In [1]:
states_d = {'schleswig':'SH', 'hamburg':'HH', 'berlin':'BE', 'bayern':'BY', 
            'niedersachsen': 'NI', 'bremen': 'HB', 
            'nordrhein':'NW', 'hessen':'HE', 'rheinland':'RP', 'baden':'BW', 
            'saarland': 'SL', 'brandenburg':'BB', 'mecklenburg':'MV', 'sachsen':'SN',
            'anhalt':'ST', 'thüringen':'TH', 'deutschland':'DE', 'hier':'DE'}
state_names = {'SH':'Schleswig-Hostein', 'HH':'Hamburg', 'BE':'Berlin', 'BY':'Bayern', 
            'NI':'Niedersachsen', 'HB':'Bremen', 
            'NW': 'Nordrhein Westphalen', 'HE':'Hessen', 'RP':'Rheinland Pfalz', 'BW':'Baden Würthenberg', 
            'SL':'Saarland', 'BB':'Brandenburg', 'MV':'Mecklenburg Vorpommern', 
            'SN': 'Sachsen', 'ST':'Sachsen-Anhalt', 'TH':'Thüringen', 'DE':'Deutschland'}
vaccines_d = {'biontech':'biontech', 'biontec':'biontech', 
              'moderna':'moderna', 
              'janssen':'janssen', 'jansen':'janssen',
              'delta':'delta',
              'astraZeneca':'astraZeneca', 'astra':'astraZeneca', 'zeneca':'astraZeneca'}
vaccine_names = {'biontech':'Biontech', 'moderna':'Moderna', 'janssen':'Janssen', 'delta':'Delta',
              'astraZeneca':'Astra Zeneca'}
import requests
def dialogmanager():
    input_s = do_input()
    input_s = normalize(input_s)
    while input_s and input_s != 'fertig':
        semantics = semantic(input_s)
        result = data(semantics)
        out_string = output(semantics, result)
        print(out_string)
        input_s = do_input()

def normalize(in_s):
    return in_s.lower()

def do_input():
    return input('> ')

def semantic(input_s):
    semantics = {'state':'', 'vaccine':''}
    for key in states_d.keys():
        if key in input_s:
            semantics['state'] = states_d[key]
    for key in  vaccines_d.keys():
        if key in input_s:
            semantics['vaccine'] =  vaccines_d[key]
    return semantics

# expects semantics: semantics[0] == bundesland, semantics[1] == impfstoff 
vaccinations = requests.get('https://api.corona-zahlen.org/vaccinations')
def data(semantics):
    s = semantics['state']
    v = semantics['vaccine']
    if s: # state given
        if s != 'DE':
            if v: # and vaccine given
                vacc_number = vaccinations.json()["data"]["states"][s]['vaccination'][v]
            else: # all vaccines for state
                vacc_number = vaccinations.json()["data"]["states"][s]['vaccinated']
        else:
            if v: # and vaccine given
                vacc_number = vaccinations.json()["data"]['vaccination'][v]
            else: # all vaccines for Germany
                vacc_number = vaccinations.json()['data']['vaccinated']
    else: # no state
        if v: # but vaccine
            vacc_number = vaccinations.json()["data"]['vaccination'][v]
        else: # nothing given
            vacc_number = None
    return vacc_number

def output(semantics, results):
    ret = ''
    s = semantics['state']
    v = semantics['vaccine']
    if s: # state given
        s = state_names[semantics['state']]
        if v: # and vaccine given
            v = vaccine_names[semantics['vaccine']]
            ret = 'Die Impfungen für {} mit {} sind {}'.format(s, v, results)
        else: # all vaccines for state
            ret = 'Die Impfungen für {} sind {}'.format(s, results)
    else: # no state
        if v: # but vaccine
            v = vaccine_names[semantics['vaccine']]
            ret = 'Die Impfungen in Deutschland mit {} sind {}'.format(v, results)
        else: # nothing given
            ret = 'Darauf weiß ich keine Antwort'
    return ret



In [2]:
dialogmanager()

> wie viel in bayern
Die Impfungen für Bayern sind 5596410
> moderna in hessen
Die Impfungen für Hessen mit Moderna sind 223926
> juhu
Darauf weiß ich keine Antwort
> 
